# Nonlinear Cart-Pole


The model is a version of the cart-pole introduced by Barto, Sutton, and Anderson in [1].

## Model

The dynamics of the cart-pole system are described as follows:

$$
\begin{aligned}
\ddot{x} &= \dfrac{u + mlω^2sin(θ)}{m_t} - \dfrac{ml(g sin(θ)- cos(θ))
            (\dfrac{u + mlω^2sin(θ)}{m_t})}{l(\dfrac{4}{3} - \dfrac{m cos(θ)^2}{m_t})}
            \dfrac{cos(θ)}{m_t}    \\
\ddot{θ} &= \dfrac{(g sin(θ)- cos(θ))(\dfrac{u + mlω^2sin(θ)}{m_t})}{l(\dfrac{4}{3}
            - \dfrac{m cos(θ)^2}{m_t})} \dfrac{cos(θ)}{m_t}
\end{aligned}
$$
where ``u ∈ {−10,10}`` is the input force, which either pushes the cart left
or right, ``g = 9.8`` is gravity, ``m = 0.1`` is the pole’s mass, ``l = 0.5``
is half the pole’s length , ``m_t = 1.1`` is the total mass, ``x`` is the
position of the cart, θ is the angle of the pendulum with respect to the
positive y-axis, ``v = \dot{x}`` is the linear velocity of the cart, and
``ω = \dot{θ}`` is the angular velocity of the pendulum. The controller has four
inputs ``(x, \dot{x}, \theta, ̇\dot{\theta})``, four layers with ``[24, 48, 12, 2]`` neurons
respectively, and two outputs. The two outputs are then compared, and the
input sent to the plant depends on which output index has the greatest value.
 Thus, as an example if ``\textrm{output}_1 > \textrm{output}_2`` then the input force supplied
to the plant is 10.  However if ``\textrm{output}_1 < \textrm{output}_2`` then the input supplied
to the plant is -10.

In [1]:
using NeuralNetworkAnalysis

@taylorize function cartpole!(du, u, p, t)
    local f, m, l, mt, g = 10, 0.1, 0.5, 1.1, 9.8
    sinθ = sin(u[3])
    cosθ = cos(u[3])
    aux = (f + m*l*u[4]^2*sinθ) / mt
    aux2 = l*(4/3 - m*cosθ^2/mt)
    aux3 = (g*sinθ- cosθ) * aux / aux2
    aux4 = cosθ/mt
    aux5 = m*l*aux3

    du[1] = u[2]
    du[2] = aux - aux5 * aux4
    du[3] = u[4]
    du[4] = aux3 * aux4

    return du
end

define the initial-value problem

In [2]:
X₀ = Hyperrectangle(low=[-0.05, -0.05, -0.05, -0.05], high=[0.05, 0.05, 0.05, 0.05])

prob = @ivp(x' = cartpole!(x), dim: 4, x(0) ∈ X₀)

InitialValueProblem{BlackBoxContinuousSystem{typeof(Main.##1443.cartpole!)}, Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}}(BlackBoxContinuousSystem{typeof(Main.##1443.cartpole!)}(Main.##1443.cartpole!, 4), Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}([0.0, 0.0, 0.0, 0.0], [0.05, 0.05, 0.05, 0.05]))

solve it

In [3]:
# sol = solve(prob, T=1.0, alg=TMJets(maxsteps=20_000, abstol=1e-10));

## Specifications

For this benchmark, the verification objective is to demonstrate that the
pole will eventually reach the upward position and that it will remain there.
 In other words, the goal is to achieve a value of ``θ = 0`` and stay there.
 Some other specifications to be met are, for at least 12 seconds,
``x ∈ [-2.4,2.4]`` and ``θ ∈ [-15,15]`` degrees. The initial conditions for
all state variables were chosen uniformly at random between ``[-0.05, 0.05]``.

## Results

In [4]:
using MAT
path = joinpath(@modelpath("Cart-Pole", "CartPole_Controller.mat"))
controller = read_nnet_mat(path, key="controller")

Network(NeuralVerification.Layer[NeuralVerification.Layer{NeuralVerification.ReLU, Float64}([0.2523224651813507 -0.24556699395179749 -1.150375485420227 -0.7059182524681091; -0.44626951217651367 -0.0048289792612195015 -1.1998661756515503 -0.7547398209571838; … ; 0.6441559195518494 0.5864951610565186 0.47801515460014343 0.2738604247570038; 0.4826578199863434 0.413881778717041 0.7180394530296326 -0.012315474450588226], [-0.006305568851530552, 0.1976671665906906, -0.17124159634113312, 0.5209460854530334, 0.5022096633911133, 0.6057475209236145, 0.8231613039970398, 0.8142926692962646, 0.6210646033287048, 0.49808186292648315  …  -0.41809532046318054, 0.6946982741355896, 0.18341603875160217, 0.6705408692359924, 0.40656644105911255, 0.6024280190467834, 0.23196551203727722, 0.5509945154190063, 0.5545315742492676, 0.41871100664138794], NeuralVerification.ReLU()), NeuralVerification.Layer{NeuralVerification.ReLU, Float64}([0.1779315322637558 0.1859981119632721 … 0.12950178980827332 -0.355859071016

## References

[^BSA83]: A. G. Barto, R. S. Sutton, and C. W. Anderson. *Neuronlike adaptive
          elements that can solve difficult learning control problems.*
          [IEEE Transactions on Systems, Man, and Cybernetics, SMC-13(5):834–846,
          Sep. 1983](https://ieeexplore.ieee.org/abstract/document/6313077).